In [2]:
import os
import calendar
from PIL import Image
from PIL.ExifTags import TAGS
import cv2
from datetime import datetime

def get_taken_date(path):
    try:
        image = Image.open(path)
        exif_data = image._getexif()
        if exif_data:
            date_fields = ['DateTimeOriginal', 'DateTimeDigitized', 'DateTime']
            for tag, value in exif_data.items():
                decoded_tag = TAGS.get(tag, tag)
                if decoded_tag in date_fields:
                    try:
                        date_obj = datetime.strptime(value, "%Y:%m:%d %H:%M:%S")
                        print(f"[✓] Taken: {date_obj} - File: {os.path.basename(path)}")
                        return date_obj
                    except ValueError:
                        print(f"[!] Invalid date format: {value} - File: {os.path.basename(path)}")
        print(f"[!] No valid EXIF date: {os.path.basename(path)}")
    except Exception as e:
        print(f"[ERROR] Failed to read EXIF: {path} - Error: {e}")
    return None

def get_images_sorted_by_date(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(folder_path, filename)
            date_taken = get_taken_date(path)
            if date_taken:
                images.append((path, date_taken))
    
    # ✅ 按月、日排序，不管年份
    images.sort(key=lambda x: (x[1].month, x[1].day))

    print("\n[✔] Sorted image list:")
    for path, dt in images:
        print(f"→ {dt.strftime('%B %d')} - {os.path.basename(path)}")

    return images

def create_video_with_month_labels(images, output_path, fps=1):
    if not images:
        print("[!] No images to process.")
        return

    frame = cv2.imread(images[0][0])
    height, width, _ = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    month_names = list(calendar.month_name)[1:]  # ['January', ..., 'December']

    for img_path, date_taken in images:
        frame = cv2.imread(img_path)
        resized_frame = cv2.resize(frame, (width, height))

        # Label with English month name
        month_text = month_names[date_taken.month - 1]
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 3
        color = (0, 255, 255)
        thickness = 5
        position = (50, 100)

        cv2.putText(resized_frame, month_text, position, font, font_scale, color, thickness, cv2.LINE_AA)
        video.write(resized_frame)

    video.release()
    print(f"\n[✔] Video saved to: {output_path}")

def main():
    folder_path = r"C:\Users\weihua\Desktop\vmo\Garden"  # 修改成你的图片路径
    output_path = 'output_video.mp4'

    images = get_images_sorted_by_date(folder_path)
    create_video_with_month_labels(images, output_path, fps=1)

if __name__ == "__main__":
    main()


[!] No valid EXIF date: 173216.png
[!] No valid EXIF date: 173534.png
[!] No valid EXIF date: 185238.png
[!] No valid EXIF date: 185353.png
[!] No valid EXIF date: 185515.png
[!] No valid EXIF date: 185622.png
[✓] Taken: 2018-03-19 15:49:50 - File: IMG_0309.JPG
[✓] Taken: 2018-05-25 07:23:05 - File: IMG_0860.JPG
[✓] Taken: 2018-05-25 07:27:19 - File: IMG_0867.JPG
[✓] Taken: 2018-05-25 10:53:17 - File: IMG_0871.JPG
[✓] Taken: 2018-05-28 14:01:56 - File: IMG_0880.JPG
[✓] Taken: 2018-05-28 14:02:20 - File: IMG_0882.JPG
[✓] Taken: 2018-05-28 14:04:17 - File: IMG_0889.JPG
[✓] Taken: 2019-05-11 17:28:09 - File: IMG_1435.JPG
[✓] Taken: 2019-05-11 17:29:47 - File: IMG_1438.JPG
[✓] Taken: 2019-05-20 15:46:42 - File: IMG_1449.JPG
[✓] Taken: 2019-05-20 15:47:43 - File: IMG_1451.JPG
[✓] Taken: 2019-07-27 09:13:17 - File: IMG_1579.JPG
[✓] Taken: 2022-03-12 08:58:15 - File: IMG_3244.JPG
[✓] Taken: 2022-04-05 13:42:16 - File: IMG_3252.JPG
[✓] Taken: 2022-05-18 10:04:59 - File: IMG_3430.JPG
[✓] Taken: